In [156]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import fuzzywuzzy for text matching
from fuzzywuzzy import fuzz, process

# Import machine learning libraries
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error,r2_score
from scipy import stats

In [157]:
df1 = pd.read_csv("data/Bengaluru_House_Data.csv")
df1.head(10)

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00
5,Super built-up Area,Ready To Move,Whitefield,2 BHK,DuenaTa,1170,2.0,1.0,38.00
6,Super built-up Area,18-May,Old Airport Road,4 BHK,Jaades,2732,4.0,NaN,204.00
7,Super built-up Area,Ready To Move,Rajaji Nagar,4 BHK,Brway G,3300,4.0,NaN,600.00
8,Super built-up Area,Ready To Move,Marathahalli,3 BHK,NaN,1310,3.0,1.0,63.25
9,Plot Area,Ready To Move,Gandhi Bazar,6 Bedroom,NaN,1020,6.0,NaN,370.00


In [158]:
df2 = df1.drop(['society', 'area_type'], axis='columns')
df2.head()

,availability,location,size,total_sqft,bath,balcony,price
0,19-Dec,Electronic City Phase II,2 BHK,1056,2.0,1.0,39.07
1,Ready To Move,Chikka Tirupathi,4 Bedroom,2600,5.0,3.0,120.00
2,Ready To Move,Uttarahalli,3 BHK,1440,2.0,3.0,62.00
3,Ready To Move,Lingadheeranahalli,3 BHK,1521,3.0,1.0,95.00
4,Ready To Move,Kothanur,2 BHK,1200,2.0,1.0,51.00


In [159]:
def convert_sqft_to_num(x):
    # Dictionary mapping units to their conversion factors to square feet
    conversion_factors = {
        'Sq. Meter': 10.7639,
        'Sq. Yard': 9,
        'Cent': 435.6,
        'Acre': 43560
    }

    # Check for unit and convert
    for unit, factor in conversion_factors.items():
        if unit in x:
            number = float(x.split(unit)[0].strip())
            return round(number * factor, 2)

    # Handle ranges of values
    tokens = x.split('-')
    if len(tokens) == 2:
        return round((float(tokens[0]) + float(tokens[1])) / 2, 2)

    # Handle direct conversion to float
    try:
        return round(float(x), 2)
    except ValueError:
        return None  # Return None if conversion fails

In [160]:
df2['total_sqft'] = df2['total_sqft'].apply(convert_sqft_to_num)
df2.head(20)

,availability,location,size,total_sqft,bath,balcony,price
0,19-Dec,Electronic City Phase II,2 BHK,1056.0,2.0,1.0,39.07
1,Ready To Move,Chikka Tirupathi,4 Bedroom,2600.0,5.0,3.0,120.00
2,Ready To Move,Uttarahalli,3 BHK,1440.0,2.0,3.0,62.00
3,Ready To Move,Lingadheeranahalli,3 BHK,1521.0,3.0,1.0,95.00
4,Ready To Move,Kothanur,2 BHK,1200.0,2.0,1.0,51.00
5,Ready To Move,Whitefield,2 BHK,1170.0,2.0,1.0,38.00
6,18-May,Old Airport Road,4 BHK,2732.0,4.0,NaN,204.00
7,Ready To Move,Rajaji Nagar,4 BHK,3300.0,4.0,NaN,600.00
8,Ready To Move,Marathahalli,3 BHK,1310.0,3.0,1.0,63.25
9,Ready To Move,Gandhi Bazar,6 Bedroom,1020.0,6.0,NaN,370.00


In [161]:
# Extract number of bedrooms from size
df2['bhk'] = df2['size'].apply(lambda x: int(str(x).split(' ')[0]) if isinstance(x, str) else None)
df2.head()

,availability,location,size,total_sqft,bath,balcony,price,bhk
0,19-Dec,Electronic City Phase II,2 BHK,1056.0,2.0,1.0,39.07,2.0
1,Ready To Move,Chikka Tirupathi,4 Bedroom,2600.0,5.0,3.0,120.00,4.0
2,Ready To Move,Uttarahalli,3 BHK,1440.0,2.0,3.0,62.00,3.0
3,Ready To Move,Lingadheeranahalli,3 BHK,1521.0,3.0,1.0,95.00,3.0
4,Ready To Move,Kothanur,2 BHK,1200.0,2.0,1.0,51.00,2.0


In [162]:
df3 = df2.dropna(subset=['location', 'size', 'total_sqft', 'bhk'])
df3.isnull().sum()

availability      0
location          0
size              0
total_sqft        0
bath             57
balcony         592
price             0
bhk               0
dtype: int64

In [163]:
df4 = df3[df3.bhk < 10]
df4['bhk'].value_counts()

bhk
2.0    5528
3.0    4856
4.0    1417
1.0     654
5.0     355
6.0     221
7.0     100
8.0      89
9.0      53
Name: count, dtype: int64

In [164]:
df4['balcony'] = df4['balcony'].fillna(0)
df5 = df4.drop('size', axis='columns')
df5.head()

C:\Users\91931\AppData\Local\Temp\ipykernel_7280\3739498709.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['balcony'] = df4['balcony'].fillna(0)


,availability,location,total_sqft,bath,balcony,price,bhk
0,19-Dec,Electronic City Phase II,1056.0,2.0,1.0,39.07,2.0
1,Ready To Move,Chikka Tirupathi,2600.0,5.0,3.0,120.00,4.0
2,Ready To Move,Uttarahalli,1440.0,2.0,3.0,62.00,3.0
3,Ready To Move,Lingadheeranahalli,1521.0,3.0,1.0,95.00,3.0
4,Ready To Move,Kothanur,1200.0,2.0,1.0,51.00,2.0


In [165]:
df5['bath'] = df5.apply(lambda row: row['bhk'] if pd.isnull(row['bath']) else row['bath'], axis=1)
df5.isnull().sum()

availability    0
location        0
total_sqft      0
bath            0
balcony         0
price           0
bhk             0
dtype: int64

In [166]:
df6 = df5[df5.price < 1000]
df6.head()


,availability,location,total_sqft,bath,balcony,price,bhk
0,19-Dec,Electronic City Phase II,1056.0,2.0,1.0,39.07,2.0
1,Ready To Move,Chikka Tirupathi,2600.0,5.0,3.0,120.00,4.0
2,Ready To Move,Uttarahalli,1440.0,2.0,3.0,62.00,3.0
3,Ready To Move,Lingadheeranahalli,1521.0,3.0,1.0,95.00,3.0
4,Ready To Move,Kothanur,1200.0,2.0,1.0,51.00,2.0


In [167]:
# # Calculate price per square foot and round to 2 decimal points
# df6['price_per_sqft'] = ((df6['price'] * 100000) / df6['total_sqft']).round(2)

# # Display the first few rows of the dataframe
# df6.head()

In [168]:
# df6['availability_binary'] = df6['availability'].apply(lambda x: 1 if x == 'Ready To Move' else 0)
# print(df6['availability_binary'].value_counts())
# df6.head()

In [169]:
df7 = df6.drop(['availability'],axis='columns')
# .drop('availability', axis='columns')
# df7.head()


In [170]:
df7.head()

,location,total_sqft,bath,balcony,price,bhk
0,Electronic City Phase II,1056.0,2.0,1.0,39.07,2.0
1,Chikka Tirupathi,2600.0,5.0,3.0,120.00,4.0
2,Uttarahalli,1440.0,2.0,3.0,62.00,3.0
3,Lingadheeranahalli,1521.0,3.0,1.0,95.00,3.0
4,Kothanur,1200.0,2.0,1.0,51.00,2.0


In [171]:
# def standardize_location(name):
#     return name.lower().strip().replace(' ', '')

# df7['standardized_location'] = df7['location'].apply(standardize_location)

In [172]:
# exact_duplicates = df7[df7.duplicated(['standardized_location'], keep=False)]


In [173]:
# # Find near duplicates
# def find_near_duplicates(locations, threshold=90):
#     near_duplicates = {}
#     for loc in locations:
#         matches = process.extract(loc, locations, scorer=fuzz.token_sort_ratio)
#         near_duplicates[loc] = [match for match in matches if match[1] >= threshold and match[0] != loc]
#     return near_duplicates

# unique_locations = df7['standardized_location'].unique()
# near_duplicates = find_near_duplicates(unique_locations, threshold=95)
# near_duplicates = {key: value for key, value in near_duplicates.items() if value}

In [174]:
# # replace duplicate values in df7 as per the duplicate values found in near_duplicates_df

# # Create a dictionary mapping near-duplicates to their most frequent representation
# replacement_dict = {}
# for location, duplicates in near_duplicates.items():
#   all_occurrences = [location] + [d[0] for d in duplicates]
#   most_frequent = max(set(all_occurrences), key=all_occurrences.count)
#   replacement_dict[location] = most_frequent
#   for duplicate in duplicates:
#     replacement_dict[duplicate[0]] = most_frequent

In [175]:
# df7['standardized_location'] = df7['standardized_location'].replace(replacement_dict)

In [176]:
df8 = df7
# .drop('location', axis='columns')
df8.head()


,location,total_sqft,bath,balcony,price,bhk
0,Electronic City Phase II,1056.0,2.0,1.0,39.07,2.0
1,Chikka Tirupathi,2600.0,5.0,3.0,120.00,4.0
2,Uttarahalli,1440.0,2.0,3.0,62.00,3.0
3,Lingadheeranahalli,1521.0,3.0,1.0,95.00,3.0
4,Kothanur,1200.0,2.0,1.0,51.00,2.0


In [177]:
# Log transformation for large values
df8['log_total_sqft'] = np.log1p(df8['total_sqft'])
df8['log_price'] = np.log1p(df8['price'])


# Standardization
scaler = StandardScaler()
df8['std_total_sqft'] = scaler.fit_transform(df8[['total_sqft']])
df8['std_price'] = scaler.fit_transform(df8[['price']])


# Normalization
normalizer = MinMaxScaler()
df8['norm_total_sqft'] = normalizer.fit_transform(df8[['total_sqft']])
df8['norm_price'] = normalizer.fit_transform(df8[['price']])



In [178]:
# # Calculate 1st and 99th percentiles
# low_percentile = df8['log_price_per_sqft'].quantile(0.01)
# high_percentile = df8['log_price_per_sqft'].quantile(0.99)

# # Remove outliers
# df = df8[(df8['log_price_per_sqft'] >= low_percentile) & (df8['log_price_per_sqft'] <= high_percentile)]

# 2. Remove total sqft outliers

# Standardize total_sqft
df8['standardized_total_sqft'] = stats.zscore(df['total_sqft'])

# Remove points beyond 3 standard deviations
df8 = df8[np.abs(df8['standardized_total_sqft']) <= 3]

# Reset index after removing rows
df8 = df8.reset_index(drop=True)

print(f"Shape of DataFrame after outlier removal: {df8.shape}")

Shape of DataFrame after outlier removal: (12670, 13)


In [179]:
# Filter rows based on conditions

df9 = df8[~((df8['bath'] >= 4) & (df8['total_sqft'] < 2000))]
df9.shape


(11857, 13)

In [180]:
df10 = df9[~(df9['total_sqft'] < 300)]
df10.shape


(11853, 13)

In [181]:
# mean_prices = df10.groupby('standardized_location')['price'].mean()

# # Map the mean prices to the original DataFrame
# df10['location_encoded'] = df10['standardized_location'].map(mean_prices)

# print(df10)

In [182]:
# # Calculate the global mean of the target variable
# global_mean = df10['price'].mean()

# # Define the smoothing parameter
# m = 3

# # Apply smoothing to the target encoding
# def smooth_mean_encoding(col, target, m, global_mean):
#     agg = df10.groupby(col)[target].agg(['mean', 'count'])
#     mean_encoded = (agg['count'] * agg['mean'] + m * global_mean) / (agg['count'] + m)
#     return mean_encoded

# # Apply the smooth mean encoding to the 'standardized_location' column
# mean_encoded = smooth_mean_encoding('standardized_location', 'price', m, global_mean)
# df10['location_encoded'] = df10['standardized_location'].map(mean_encoded)

# print(df10)


In [183]:
df10.head()

,location,total_sqft,bath,balcony,price,bhk,log_total_sqft,log_price,std_total_sqft,std_price,norm_total_sqft,norm_price,standardized_total_sqft
0,Electronic City Phase II,1056.0,2.0,1.0,39.07,2.0,6.963190,3.690628,-0.051716,-0.637914,0.000807,0.031479,-0.565018
1,Chikka Tirupathi,2600.0,5.0,3.0,120.00,4.0,7.863651,4.795791,0.056086,0.131356,0.001989,0.113475,1.339606
2,Uttarahalli,1440.0,2.0,3.0,62.00,3.0,7.273093,4.143135,-0.024905,-0.419955,0.001101,0.054711,-0.091329
3,Lingadheeranahalli,1521.0,3.0,1.0,95.00,3.0,7.327781,4.564348,-0.019250,-0.106278,0.001163,0.088146,0.008590
4,Kothanur,1200.0,2.0,1.0,51.00,2.0,7.090910,3.951244,-0.041662,-0.524514,0.000918,0.043566,-0.387385


In [184]:
# drop the following columns standardized_location	log_total_sqft	log_price	std_total_sqft	std_price	std_price_per_sqft	norm_total_sqft	norm_price	norm_price_per_sqft	log_price_per_sqft	standardized_total_sqft

# df11 = df10.drop(['standardized_location', 'log_total_sqft', 'log_price', 'std_total_sqft',
#                    'std_price', 'std_price_per_sqft', 'norm_total_sqft', 'norm_price',
#                    'norm_price_per_sqft', 'log_price_per_sqft', 'standardized_total_sqft'], axis='columns')
# df11.head(30)

df11 = df10.drop([ 'log_total_sqft', 'log_price', 'std_total_sqft',
                   'std_price', 'norm_total_sqft', 'norm_price',
                    'standardized_total_sqft'], axis='columns')
# df11.head(30)



In [185]:
df11.shape

(11853, 6)

In [186]:


# Save DataFrame to CSV with a relative path
df11.to_csv('data/cleaned.csv', index=False)
